In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import urllib.request
import re
import pandas as pd
import datetime
import os
import getpass

In [6]:
# 학위논문 수집 프로그램
# 크롤링 실행방법
# 아래 모든 사용자 정의 함수들을 실행시킨 후
# 키워드와 추출할 논문페이지수를 입력하고 이 셀을 실행시킵니다.
# 한 페이지당 10개의 논문이 있으므로 10을 입력하면 100개의 논문이 수집됩니다.
get_link("연관규칙분석", 10)

0
['연관 규칙과 분류 규칙을 이용한 은행 고객의 연체 성향 분석에 관한 연구 : 신용 카드 연체 중심으로 ', ' (The) Study on the analysis of propensity to delinquency of bank customer using association and classification rules : with focus on credit card delinquency cases']
신수일
1
['연관규칙(association rules)을 이용한 소비자의 식품구매행태 분석']
신여진
2
['단백질 기능 분석을 위한 연관 규칙 탐사 ', ' Association Rule Discovery for Protein Function Analysis']
김정자
3
['음식과 와인 조화와 와인선택속성에 관한 연구 : BWS, 연관규칙 분석방법을 이용하여 ', ' A Study on Food and Wine Pairing and Wine Selection Attributes : Using BWS, Association Rules Analysis']
한경은
4
['연관규칙 분석기법을 활용한 공동주택 시공단계의 결함발생규칙 규명 ', ' Identification on Defect Generating Rule of Multi-Housing Project in Construction Phase using Association Rule Analysis']
박정은
5
['텍스트 마이닝과 연관 규칙 분석을 통합한 고객행동분석 방법론 연구 ', ' Development of customer behavior analysis methodology integrating text mining and association rule mining']
모성훈
6
['구문적 패턴이 결합된 정량적 연관 규칙 탐사 시스템을 이용한 시계열 분석 ', ' Time series analysis using syntactic pattern based quantitative associa

80
['연관규칙을 이용한 의료데이터 마이닝 ', ' Mining association rules from medical records']
임준호
81
['연관규칙 기법을 이용한 학생 정보 마이닝 시스템(SIMS)의 설계 및 구현 ', ' Design and Implementation Student Information Mining System (SIMS) using Association Rules']
공현선
82
['연관규칙을 이용한 미토콘드리아 DNA 분류학습']
황효선
83
['연관도 분석에 기반한 능동적 교차 도메인 추천 모델에 관한 연구 ', ' A study on adaptive cross-domain recommendation model based on association analysis']
배은영
84
['연관규칙 마이닝을 활용한 데이터베이스 설계 자동화 기법 ', ' Semi-Automated Conceptual Data Modeling Using Association Rule Mining']
손윤호
85
['가중치 연관규칙 탐사 알고리즘을 이용한 자동 문헌 분류 ', ' (An) Automatic document classification using weight association rule algorithm']
강미나
86
['연관규칙 탐사와 사례기반 추론 기법에 의한 유무선 통합 지능형 검색 에이전트 시스템의 구현에 관한 연구 ', ' (A)study on the implementation of convergence intelligent search agent system using association rules and case-based reasoning techniques']
하창승
87
['연관 규칙과 협동적 추천의 통합에 의한 추천 시스템의 성능 향상 ', ' Performance improvement of a recommendation system by combining collaborative recommendation wi

In [2]:
def get_URL(keyword, page):
    #url_before_page="http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%A7%88%EC%9D%B4%EB%8B%9D&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    #url_after_page="&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%A7%88%EC%9D%B4%EB%8B%9D"
    
    ascii_keyword = str(keyword.encode('utf-8')).replace("b'","").replace("\\x", "%").upper().replace("'","")

    url_before_page="http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery={}&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=".format(ascii_keyword)
    url_after_page="&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query={}".format(ascii_keyword)
 
    
    URL=url_before_page + page + url_after_page
    return URL

In [3]:
def get_link(csv_name, page_num):
    
    for i in range(page_num):
        current_page = i*10
        #URL = get_URL(str(current_page))
        URL = get_URL(csv_name, str(current_page))
        source_code_from_URL = urllib.request.urlopen(URL)
        soup = bs(source_code_from_URL, 'lxml', from_encoding= 'utf-8')
        #print(current_page+1)
        for j in range(10):
            paper_link= soup.select('li > div.cont > p.title > a')[j]['href']
            paper_url = "http://riss.or.kr" + paper_link
            print(current_page+j)
            reference_data = get_reference(paper_url)
            save_csv(csv_name, reference_data)


In [4]:
def get_reference(URL):
    #driver_path = "C:\Temp\chromedriver"
    #driver = webdriver.Chrome(driver_path, options=webdriver.ChromeOptions().add_argument("headless"))
    driver = webdriver.Chrome("C:\\Temp\\chromedriver\\chromedriver.exe", options=webdriver.ChromeOptions().add_argument("headless"))
    driver.get(URL)

    html = driver.page_source
    soup = bs(html, "html.parser")
                               
    title = soup.find("h3", "title")
    title_txt = title.get_text("", strip=True).split("=")
    print(title_txt)
    title_kor = re.sub("\n\b", "", str(title_txt[0]).strip())
    try:
        title_eng = str(title_txt[1]).strip()
    except IndexError:
        title_eng = ""
                               
    txt_box = []
    for text in soup.find_all("div", "text"):
        txt = text.get_text("" , strip=True)
        txt_box.append(txt)
    
    try:
        txt_kor = txt_box[1]
    except IndexError:
        txt_kor = ""
    #print(txt_kor)
    try:
        txt_eng = txt_box[3]
    except IndexError:
        txt_eng = ""
    #print(txt_eng)
        
    detail_box = []
    #detail_info = soup.select("#soptionview > div > div.thesislnfo > div.infoDetail.on > div.infoDetailL > ul > li > div > p")
    detail_info = soup.select("#soptionview > div > div.thesisInfo > div.infoDetail.on > div.infoDetailL > ul > li > div > p")
                                  
    for detail in detail_info:
        detail_content = detail.get_text("" , strip=True)
        detail_wrap = []
        detail_wrap.append(detail_content)
        detail_box.append(detail_wrap)
            
    author = ",".join(detail_box[0])
    print(author)
    book = (
        "".join(detail_box[2] + detail_box[3]).replace("\n", "").replace("\t","").replace(" ","") 
        + " p." 
        + "".join(detail_box[-2])
    )
            
    keyword = ",".join(detail_box[5])

    reference_data = pd.DataFrame(
        {
            "저자" : [author],
            "국문 제목" : [title_kor],
            "영문 제목" : [title_eng],
            "학위논문사항" : [book],
            "핵심어" : [keyword],
            "국문 요약" : [txt_kor],
            "영문 요약" : [txt_eng],
            "링크" : [URL],
        }
    )
                
    driver.close()
    
    return reference_data

In [5]:
def save_csv(csv_path, data):
    #csv=csv_path.replace("/","\\")
    csv_file = "C:\\Temp\\{}_학위논문.csv".format(csv_path)  
    #print(csv_file)
    if os.path.isfile(csv_file):
        data.to_csv(csv_file, mode='a',header=False, index=False)
    else:
        data.to_csv(csv_file, mode='w',header=True, index=False)
